In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for data visualization
import seaborn as sns # for statistical data visualization
from datetime import datetime

In [10]:
#pip install datasets

In [11]:
import numpy as np
import torch
import torch.utils.data
import scipy
import torch.nn.functional as F
#import datasets
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
#datasets.logging.set_verbosity_error()

In [12]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

In [13]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam

In [14]:
import tensorflow
tensorflow.random.set_seed(0)
np.random.seed(0)

In [15]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dropout


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
df = pd.read_csv('drive/My Drive/thesis/dataset/final_dataset_with_all_stock.csv [Final one]')

In [18]:
df = df.drop(df.columns[0],axis=1)

In [19]:
#df

In [20]:
df = df.sort_values(by=['stock_symbol', 'Date'])

In [21]:
df

,stock_symbol,Date,Open,Close,High,Low,Volume,macd,tweet_sentiment_count_0,tweet_sentiment_count_1,...,Tuesday,Wednesday,tweet_0_pct,tweet_1_pct,tweet_2_pct,stocktwit_0_pct,stocktwit_1_pct,news_0_pct,news_1_pct,news_2_pct
0,AAPL,2022-03-01,164.699997,163.199997,166.600006,161.970001,83474400,-0.760515,164,125,...,1,0,0.340956,0.259875,0.399168,0.382781,0.617219,0.666667,0.333333,0.000000
1,AAPL,2022-03-02,164.389999,166.559998,167.360001,162.949997,79724800,-0.470632,69,200,...,0,1,0.161972,0.469484,0.368545,0.265512,0.734488,1.000000,0.000000,0.000000
2,AAPL,2022-03-03,168.470001,166.229996,168.910004,165.550003,76678400,-0.274369,103,160,...,0,0,0.213251,0.331263,0.455487,0.339372,0.660628,0.000000,1.000000,0.000000
3,AAPL,2022-03-08,158.820007,157.440002,162.880005,155.800003,131148300,-0.797608,158,503,...,1,0,0.150907,0.480420,0.368672,0.362690,0.637310,0.021277,0.808511,0.170213
4,AAPL,2022-03-09,161.479996,162.949997,163.410004,159.410004,91454900,-0.532939,84,224,...,0,1,0.156716,0.417910,0.425373,0.259504,0.740496,0.000000,0.777778,0.222222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3794,WMT,2022-12-21,144.320007,145.179993,145.940002,144.259995,4575500,-1.058663,14,23,...,0,1,0.186667,0.306667,0.506667,0.170732,0.829268,0.166667,0.666667,0.166667
3795,WMT,2022-12-22,144.770004,143.479996,144.979996,141.929993,4360800,-0.992514,9,33,...,0,0,0.105882,0.388235,0.505882,0.258065,0.741935,0.055556,0.777778,0.166667
3796,WMT,2022-12-27,144.039993,143.809998,144.449997,143.190002,3348200,-0.770568,19,25,...,1,0,0.250000,0.328947,0.421053,0.437500,0.562500,0.166667,0.416667,0.416667
3797,WMT,2022-12-28,143.470001,141.289993,143.669998,140.910004,5082100,-0.819899,16,24,...,0,1,0.200000,0.300000,0.500000,0.366667,0.633333,0.111111,0.555556,0.333333


In [22]:
def calculate_price_change(group):
    Price_Change = group['Open'].shift(-1) - group['Open']
    group['price_change'] = Price_Change
    group['Open_t+1'] = group['Open'].shift(-1)
    return group

group_df = df.groupby('stock_symbol', group_keys=False).apply(calculate_price_change)


In [23]:
def calculate_direction(group):
    Direction = group['price_change'].apply(lambda x: 1 if isinstance(x, (float, int)) and x > 0 else 0 if isinstance(x, (float, int)) and x <= 0 else None)
    group['direction'] = Direction
    return group

group_df = group_df.groupby('stock_symbol', group_keys=False).apply(calculate_direction)


In [24]:
group_df

,stock_symbol,Date,Open,Close,High,Low,Volume,macd,tweet_sentiment_count_0,tweet_sentiment_count_1,...,tweet_1_pct,tweet_2_pct,stocktwit_0_pct,stocktwit_1_pct,news_0_pct,news_1_pct,news_2_pct,price_change,Open_t+1,direction
0,AAPL,2022-03-01,164.699997,163.199997,166.600006,161.970001,83474400,-0.760515,164,125,...,0.259875,0.399168,0.382781,0.617219,0.666667,0.333333,0.000000,-0.309998,164.389999,0.0
1,AAPL,2022-03-02,164.389999,166.559998,167.360001,162.949997,79724800,-0.470632,69,200,...,0.469484,0.368545,0.265512,0.734488,1.000000,0.000000,0.000000,4.080002,168.470001,1.0
2,AAPL,2022-03-03,168.470001,166.229996,168.910004,165.550003,76678400,-0.274369,103,160,...,0.331263,0.455487,0.339372,0.660628,0.000000,1.000000,0.000000,-9.649994,158.820007,0.0
3,AAPL,2022-03-08,158.820007,157.440002,162.880005,155.800003,131148300,-0.797608,158,503,...,0.480420,0.368672,0.362690,0.637310,0.021277,0.808511,0.170213,2.659988,161.479996,1.0
4,AAPL,2022-03-09,161.479996,162.949997,163.410004,159.410004,91454900,-0.532939,84,224,...,0.417910,0.425373,0.259504,0.740496,0.000000,0.777778,0.222222,-1.279999,160.199997,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3794,WMT,2022-12-21,144.320007,145.179993,145.940002,144.259995,4575500,-1.058663,14,23,...,0.306667,0.506667,0.170732,0.829268,0.166667,0.666667,0.166667,0.449997,144.770004,1.0
3795,WMT,2022-12-22,144.770004,143.479996,144.979996,141.929993,4360800,-0.992514,9,33,...,0.388235,0.505882,0.258065,0.741935,0.055556,0.777778,0.166667,-0.730011,144.039993,0.0
3796,WMT,2022-12-27,144.039993,143.809998,144.449997,143.190002,3348200,-0.770568,19,25,...,0.328947,0.421053,0.437500,0.562500,0.166667,0.416667,0.416667,-0.569992,143.470001,0.0
3797,WMT,2022-12-28,143.470001,141.289993,143.669998,140.910004,5082100,-0.819899,16,24,...,0.300000,0.500000,0.366667,0.633333,0.111111,0.555556,0.333333,-1.330002,142.139999,0.0


In [25]:
group_df.isnull().sum()

stock_symbol                    0
Date                            0
Open                            0
Close                           0
High                            0
Low                             0
Volume                          0
macd                            0
tweet_sentiment_count_0         0
tweet_sentiment_count_1         0
tweet_sentiment_count_2         0
tweet_sentiment_mean            0
stocktwit_sentiment_count_0     0
stocktwit_sentiment_count_1     0
stocktwit_sentiment_mean        0
news_sentiment_count_0          0
news_sentiment_count_1          0
news_sentiment_count_2          0
news_sentiment_mean             0
trends                          0
Null_value_news                 0
Null_value_stocktwit            0
week_day                        0
Thursday                        0
Tuesday                         0
Wednesday                       0
tweet_0_pct                     0
tweet_1_pct                     0
tweet_2_pct                     0
stocktwit_0_pc

In [26]:
group_df = group_df.dropna()

In [27]:
group_df.shape

(3770, 37)

In [28]:
group_df.columns

Index(['stock_symbol', 'Date', 'Open', 'Close', 'High', 'Low', 'Volume',
       'macd', 'tweet_sentiment_count_0', 'tweet_sentiment_count_1',
       'tweet_sentiment_count_2', 'tweet_sentiment_mean',
       'stocktwit_sentiment_count_0', 'stocktwit_sentiment_count_1',
       'stocktwit_sentiment_mean', 'news_sentiment_count_0',
       'news_sentiment_count_1', 'news_sentiment_count_2',
       'news_sentiment_mean', 'trends', 'Null_value_news',
       'Null_value_stocktwit', 'week_day', 'Thursday', 'Tuesday', 'Wednesday',
       'tweet_0_pct', 'tweet_1_pct', 'tweet_2_pct', 'stocktwit_0_pct',
       'stocktwit_1_pct', 'news_0_pct', 'news_1_pct', 'news_2_pct',
       'price_change', 'Open_t+1', 'direction'],
      dtype='object')

# 1. price, macd, twitter, stocktwit---finished

#2. price, macd, twitter, stocktwit, news



#3. price,macd,twitter,stocktwit,news,trends---

#4. price,macd,twitter,stocktwit,trends---

#5. price,macd,news,trends---

#6. price,macd,twitter---







#7. price,macd,stocktwit---

#8. price,macd,news---

#9. price,macd,trends---

#10. price,macd---

In [29]:
data = group_df[['stock_symbol', 'Date', 'Open', 'Close', 'High', 'Low',
       'macd', 'Thursday', 'Tuesday', 'Wednesday', 'price_change', 'Open_t+1', 
       'direction', 
       'tweet_sentiment_count_0', 'tweet_sentiment_count_1', 
       'tweet_sentiment_count_2', 'tweet_sentiment_mean', 
       'stocktwit_sentiment_count_0', 'stocktwit_sentiment_count_1', 
       'stocktwit_sentiment_mean', 
       #'news_sentiment_count_0', 
       #'news_sentiment_count_1', 'news_sentiment_count_2', 
       #'news_sentiment_mean', 
       #'trends', 
       #'Null_value_news',
       'Null_value_stocktwit'
       ]].copy()

In [30]:
data.columns

Index(['stock_symbol', 'Date', 'Open', 'Close', 'High', 'Low', 'macd',
       'Thursday', 'Tuesday', 'Wednesday', 'price_change', 'Open_t+1',
       'direction', 'tweet_sentiment_count_0', 'tweet_sentiment_count_1',
       'tweet_sentiment_count_2', 'tweet_sentiment_mean',
       'stocktwit_sentiment_count_0', 'stocktwit_sentiment_count_1',
       'stocktwit_sentiment_mean', 'Null_value_stocktwit'],
      dtype='object')

In [31]:
data.shape

(3770, 21)

In [32]:
def train_val_test_split(data):
    size=int(len(data)*0.6)
    # for train data will be collected from each country's data which index is from 0-size (60%)
    x_train = data.drop(columns=['direction']).iloc[0:size] 
    # for validation data 
    rest = data.iloc[size:]

    size2 = int(len(rest)*0.3)
    x_val = rest.drop(columns=['direction']).iloc[0:size2]
    # for test data
    x_test = rest.drop(columns=['direction']).iloc[size2:] 

    y_train = data['direction'].iloc[0:size] 
    y_val = rest['direction'].iloc[0:size2]
    y_test =  rest['direction'].iloc[size2:]

    return x_train, x_val, x_test, y_train, y_val, y_test

In [33]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

def reshape_train_data(data):
  # Prepare the data
  scaler = StandardScaler()

  # Group the data by stock_symbol
  grouped = data.groupby('stock_symbol')

  X_list = []
  y_list = []
  Z_list = []

  seq_length = 9

  for stock_symbol, group in grouped:
      # Scale the data
      scaled_x = scaler.fit_transform(group.drop(['stock_symbol','Date','direction','Open_t+1'], axis=1))
      scaled_y = np.array(group['direction'])
      scaled_z = np.array(group['Open_t+1'])


      X = []
      y = []
      Z = []

      for i in range(seq_length, len(scaled_x)):
          X.append(scaled_x[i-seq_length:i])
          y.append(scaled_y[i])
          Z.append(scaled_z[i])

      X_list.append(np.array(X))
      y_list.append(np.array(y))
      Z_list.append(np.array(Z))

  # Concatenate the data for all stocks
  X_train = np.concatenate(X_list, axis=0)
  y_train = np.concatenate(y_list, axis=0)
  Open_t_1_train = np.concatenate(Z_list, axis=0)

  return X_train, y_train, Open_t_1_train




In [34]:


def reshape_val_data(data):
  # Prepare the data
  scaler = StandardScaler()

  # Group the data by stock_symbol
  grouped = data.groupby('stock_symbol')

  X_list = []
  y_list = []
  Z_list = []

  seq_length = 9

  for stock_symbol, group in grouped:
      # Scale the data
      scaled_x = scaler.fit_transform(group.drop(['stock_symbol','Date','direction','Open_t+1'], axis=1))
      scaled_y = np.array(group['direction'])
      scaled_z = np.array(group['Open_t+1'])

      X = []
      y = []
      Z = []

      for i in range(seq_length, len(scaled_x)):
          X.append(scaled_x[i-seq_length:i])
          y.append(scaled_y[i])
          Z.append(scaled_z[i])

      X_list.append(np.array(X))
      y_list.append(np.array(y))
      Z_list.append(np.array(Z))

  # Concatenate the data for all stocks
  X_val = np.concatenate(X_list, axis=0)
  y_val = np.concatenate(y_list, axis=0)
  Open_t_1_val = np.concatenate(Z_list, axis=0)

  return X_val, y_val, Open_t_1_val

  


In [35]:


def reshape_test_data(data):
  # Prepare the data
  scaler = StandardScaler()

  # Group the data by stock_symbol
  grouped = data.groupby('stock_symbol')

  X_list = []
  y_list = []
  Z_list = []

  seq_length = 9

  for stock_symbol, group in grouped:
      # Scale the data
      scaled_x = scaler.fit_transform(group.drop(['stock_symbol','Date','direction','Open_t+1'], axis=1))
      scaled_y = np.array(group['direction'])
      scaled_z = np.array(group['Open_t+1'])

      X = []
      y = []
      Z = []

      for i in range(seq_length, len(scaled_x)):
          X.append(scaled_x[i-seq_length:i])
          y.append(scaled_y[i])
          Z.append(scaled_z[i])

      X_list.append(np.array(X))
      y_list.append(np.array(y))
      Z_list.append(np.array(Z))

  # Concatenate the data for all stocks
  X_test = np.concatenate(X_list, axis=0)
  y_test = np.concatenate(y_list, axis=0)
  Open_t_1_test = np.concatenate(Z_list, axis=0)

  return X_test, y_test, Open_t_1_test

  


# Loop

In [36]:
stock=list(set(data.stock_symbol))
stock.sort()

In [37]:
#stock

In [38]:
results_dict = {}

In [39]:
labels = []

In [40]:
test_data_frame = []

In [41]:
Open_t_1_test_frame = []

In [42]:

# loop each country_Region and split the data into train and test data and train the model

for x in stock:
    stock_data = data[data['stock_symbol']==x]
    # applied the function I created above
    x_train, x_val, x_test, y_train, y_val, y_test=train_val_test_split(stock_data) 

    #combine X train and Y train as train data 
    train_data = pd.concat([x_train, y_train], axis = 1)
    val_data = pd.concat([x_val, y_val], axis = 1)
    test_data = pd.concat([x_test, y_test], axis = 1)

    test_data_frame.append(test_data)

    X_train, y_train, Open_t_1_train = reshape_train_data(train_data)
    X_val, y_val, Open_t_1_val = reshape_val_data(val_data)
    X_test, y_test, Open_t_1_test = reshape_test_data(test_data)

    Open_t_1_test_frame.append(Open_t_1_test)

    #build
    model = keras.Sequential()
    model.add(layers.LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(layers.LSTM(100, return_sequences=False))
    #model.add(layers.LSTM(256, return_sequences=False))
    #model.add(layers.LSTM(256, return_sequences=False))
    #model.add(layers.Dense(50))
    model.add(layers.Dense(25))
    model.add(layers.Dense(1,activation='sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), #validation_split=0.2, 
                    verbose=1, epochs=10, shuffle=False)

    scores = model.evaluate(X_test, y_test, verbose=2)
    print(x,"-Accuracy: %.2f%%" % (scores[1]*100))

    # Get prediction
    predicted_probabilities = model.predict(X_test)
    predicted_labels = [1 if prob >= 0.5 else 0 for prob in predicted_probabilities]
    labels.append(predicted_labels)

    Acc = accuracy_score(y_test, predicted_labels)
    class_report = classification_report(y_test, predicted_labels)
    class_report_lines = class_report.split('\n')
    precision_0 = class_report_lines[2].split()[1]
    precision_1 = class_report_lines[3].split()[1]

    print(Acc, precision_0, precision_1)

    # Add the results to the dictionary
    results_dict[x] = {'accuracy': Acc, 'precision_0': precision_0, 'precision_1': precision_1}

Epoch 1/10
3/3 [==============================] - 6s 611ms/step - loss: 0.6852 - accuracy: 0.5652 - val_loss: 0.7032 - val_accuracy: 0.5000
Epoch 2/10
3/3 [==============================] - 0s 68ms/step - loss: 0.6524 - accuracy: 0.5942 - val_loss: 0.7308 - val_accuracy: 0.3333
Epoch 3/10
3/3 [==============================] - 0s 69ms/step - loss: 0.6417 - accuracy: 0.7101 - val_loss: 0.7607 - val_accuracy: 0.3333
Epoch 4/10
3/3 [==============================] - 0s 67ms/step - loss: 0.6381 - accuracy: 0.6232 - val_loss: 0.7906 - val_accuracy: 0.3333
Epoch 5/10
3/3 [==============================] - 0s 64ms/step - loss: 0.6380 - accuracy: 0.6087 - val_loss: 0.8148 - val_accuracy: 0.3333
Epoch 6/10
3/3 [==============================] - 0s 64ms/step - loss: 0.6371 - accuracy: 0.6087 - val_loss: 0.8276 - val_accuracy: 0.3333
Epoch 7/10
3/3 [==============================] - 0s 63ms/step - loss: 0.6298 - accuracy: 0.6087 - val_loss: 0.8301 - val_accuracy: 0.3333
Epoch 8/10
3/3 [==========

1/1 [==============================] - 1s 849ms/step
0.4642857142857143 0.39 0.60
Epoch 1/10
3/3 [==============================] - 7s 569ms/step - loss: 0.6903 - accuracy: 0.5507 - val_loss: 0.6878 - val_accuracy: 0.5000
Epoch 2/10
3/3 [==============================] - 0s 37ms/step - loss: 0.6846 - accuracy: 0.5507 - val_loss: 0.6979 - val_accuracy: 0.5000
Epoch 3/10
3/3 [==============================] - 0s 35ms/step - loss: 0.6763 - accuracy: 0.5652 - val_loss: 0.7107 - val_accuracy: 0.5000
Epoch 4/10
3/3 [==============================] - 0s 45ms/step - loss: 0.6695 - accuracy: 0.5797 - val_loss: 0.7262 - val_accuracy: 0.5000
Epoch 5/10
3/3 [==============================] - 0s 37ms/step - loss: 0.6640 - accuracy: 0.6087 - val_loss: 0.7463 - val_accuracy: 0.5000
Epoch 6/10
3/3 [==============================] - 0s 41ms/step - loss: 0.6582 - accuracy: 0.6232 - val_loss: 0.7728 - val_accuracy: 0.5000
Epoch 7/10
3/3 [==============================] - 0s 42ms/step - loss: 0.6543 - acc

1/1 [==============================] - 1s 866ms/step
0.6785714285714286 0.70 0.62
Epoch 1/10
3/3 [==============================] - 5s 586ms/step - loss: 0.6982 - accuracy: 0.4928 - val_loss: 0.6580 - val_accuracy: 0.6667
Epoch 2/10
3/3 [==============================] - 0s 72ms/step - loss: 0.6821 - accuracy: 0.5507 - val_loss: 0.6935 - val_accuracy: 0.5000
Epoch 3/10
3/3 [==============================] - 0s 71ms/step - loss: 0.6726 - accuracy: 0.5507 - val_loss: 0.7037 - val_accuracy: 0.3333
Epoch 4/10
3/3 [==============================] - 0s 72ms/step - loss: 0.6621 - accuracy: 0.6377 - val_loss: 0.6829 - val_accuracy: 0.5000
Epoch 5/10
3/3 [==============================] - 0s 72ms/step - loss: 0.6500 - accuracy: 0.6667 - val_loss: 0.6701 - val_accuracy: 0.6667
Epoch 6/10
3/3 [==============================] - 0s 61ms/step - loss: 0.6394 - accuracy: 0.6522 - val_loss: 0.7005 - val_accuracy: 0.5000
Epoch 7/10
3/3 [==============================] - 0s 62ms/step - loss: 0.6265 - acc

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
3/3 [==============================] - 5s 730ms/step - loss: 0.6973 - accuracy: 0.4348 - val_loss: 0.7052 - val_accuracy: 0.3333
Epoch 2/10
3/3 [==============================] - 0s 62ms/step - loss: 0.6779 - accuracy: 0.5652 - val_loss: 0.7172 - val_accuracy: 0.3333
Epoch 3/10
3/3 [==============================] - 0s 57ms/step - loss: 0.6734 - accuracy: 0.5507 - val_loss: 0.7256 - val_accuracy: 0.3333
Epoch 4/10
3/3 [==============================] - 0s 62ms/step - loss: 0.6677 - accuracy: 0.5652 - val_loss: 0.7303 - val_accuracy: 0.3333
Epoch 5/10
3/3 [==============================] - 0s 57ms/step - loss: 0.6600 - accuracy: 0.6522 - val_loss: 0.7328 - val_accuracy: 0.3333
Epoch 6/10
3/3 [==============================] - 0s 50ms/step - loss: 0.6524 - accuracy: 0.6667 - val_loss: 0.7336 - val_accuracy: 0.3333
Epoch 7/10
3/3 [==============================] - 0s 58ms/step - loss: 0.6453 - accuracy: 0.6667 - val_loss: 0.7310 - val_accuracy: 0.3333
Epoch 8/10
3/3 [==========

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
3/3 [==============================] - 5s 515ms/step - loss: 0.6925 - accuracy: 0.5507 - val_loss: 0.6921 - val_accuracy: 0.6667
Epoch 2/10
3/3 [==============================] - 0s 47ms/step - loss: 0.6561 - accuracy: 0.6522 - val_loss: 0.7004 - val_accuracy: 0.6667
Epoch 3/10
3/3 [==============================] - 0s 42ms/step - loss: 0.6414 - accuracy: 0.6667 - val_loss: 0.7049 - val_accuracy: 0.6667
Epoch 4/10
3/3 [==============================] - 0s 40ms/step - loss: 0.6270 - accuracy: 0.6812 - val_loss: 0.7055 - val_accuracy: 0.6667
Epoch 5/10
3/3 [==============================] - 0s 37ms/step - loss: 0.6122 - accuracy: 0.6957 - val_loss: 0.7078 - val_accuracy: 0.6667
Epoch 6/10
3/3 [==============================] - 0s 33ms/step - loss: 0.5969 - accuracy: 0.7101 - val_loss: 0.7163 - val_accuracy: 0.6667
Epoch 7/10
3/3 [==============================] - 0s 34ms/step - loss: 0.5774 - accuracy: 0.7246 - val_loss: 0.7354 - val_accuracy: 0.6667
Epoch 8/10
3/3 [==========

In [43]:
results_dict

{'AAPL': {'accuracy': 0.39285714285714285,
  'precision_0': '0.50',
  'precision_1': '0.36'},
 'AMGN': {'accuracy': 0.6785714285714286,
  'precision_0': '0.79',
  'precision_1': '0.57'},
 'AXP': {'accuracy': 0.6428571428571429,
  'precision_0': '0.61',
  'precision_1': '0.70'},
 'BA': {'accuracy': 0.4642857142857143,
  'precision_0': '0.38',
  'precision_1': '0.53'},
 'CAT': {'accuracy': 0.4642857142857143,
  'precision_0': '0.39',
  'precision_1': '0.60'},
 'CRM': {'accuracy': 0.6785714285714286,
  'precision_0': '0.70',
  'precision_1': '0.62'},
 'CSCO': {'accuracy': 0.39285714285714285,
  'precision_0': '0.38',
  'precision_1': '0.40'},
 'CVX': {'accuracy': 0.5357142857142857,
  'precision_0': '0.56',
  'precision_1': '0.50'},
 'DIS': {'accuracy': 0.6428571428571429,
  'precision_0': '0.64',
  'precision_1': '0.00'},
 'DOW': {'accuracy': 0.5714285714285714,
  'precision_0': '0.53',
  'precision_1': '0.62'},
 'GS': {'accuracy': 0.6785714285714286,
  'precision_0': '0.67',
  'precisio

In [44]:
df_1 = pd.DataFrame.from_dict(results_dict)

In [45]:
df_1

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,...,MMM,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
accuracy,0.392857,0.678571,0.642857,0.464286,0.464286,0.678571,0.392857,0.535714,0.642857,0.571429,...,0.571429,0.571429,0.464286,0.428571,0.464286,0.607143,0.571429,0.428571,0.357143,0.678571
precision_0,0.50,0.79,0.61,0.38,0.39,0.70,0.38,0.56,0.64,0.53,...,0.56,0.71,0.42,0.31,0.43,0.54,0.60,0.35,0.40,0.67
precision_1,0.36,0.57,0.70,0.53,0.60,0.62,0.40,0.50,0.00,0.62,...,0.60,0.43,0.50,0.58,0.50,0.67,0.54,0.55,0.25,0.70


In [46]:
predict_labels = [x for y in labels for x in y]

In [47]:
len(predict_labels)

812

In [48]:
test_data = pd.concat(test_data_frame)

In [49]:
Open_t_1_test = [x for y in Open_t_1_test_frame for x in y]
len(Open_t_1_test)

812

In [50]:
#test_data

In [51]:
# Group the DataFrame by stock_symbol
test_group = test_data.groupby("stock_symbol")

# Remove the first 7 days of data for each stock symbol
test1 = test_group.apply(lambda x: x.iloc[9:])
test1.reset_index(drop=True, inplace=True)
#test1

In [52]:
test1['Prediction'] = predict_labels
test1['Open_t_1_test'] = Open_t_1_test

In [53]:
#test1

In [54]:
# Select the rows where prediction is equal to 1
prediction_1 = test1[test1["Prediction"] == 1].copy()

# Compute the subtraction and sum operation on the selected rows

return_rate = ((100/prediction_1["Open"])*prediction_1["Open_t_1_test"]*(1-0.001)-100*(1+0.001))/100*(1+0.001)
profit = (100/prediction_1["Open"])*prediction_1["Open_t_1_test"]*(1-0.001)-100*(1+0.001)
#result = prediction_1["Open_t_1_test"].sub(prediction_1["Open"]).sum()


# overall_return_rate = 1
# for rate in result:
#     overall_return_rate *= (1 + rate)

# # Step 3: Convert the overall return rate back to a percentage
# overall_return_percentage = (overall_return_rate - 1) * 100

# print("Overall return rate:", overall_return_percentage, "%")


# Print the result
print(return_rate, profit)

print(sum(return_rate), sum(profit))
print(return_rate.mean(), profit.mean())

0      0.003795
1     -0.021146
2      0.045340
3     -0.041530
4     -0.048259
         ...   
793    0.013588
794   -0.007885
795    0.031941
796   -0.001476
797    0.004245
Length: 354, dtype: float64 0      0.379079
1     -2.112500
2      4.529509
3     -4.148849
4     -4.821087
         ...   
793    1.357403
794   -0.787734
795    3.190936
796   -0.147420
797    0.424045
Length: 354, dtype: float64
0.34970834662720884 34.93589876395693
0.0009878766853875954 0.09868897955920036


In [55]:
test1.loc[test1['Prediction'] == 1, 'return rate'] = return_rate
test1.loc[test1['Prediction'] == 1, 'profit'] = profit

In [56]:
#test1

In [57]:
def calculate_profit(group):
  return_rate  = ((100/group["Open"])*group["Open_t_1_test"]*(1-0.001)-100*(1+0.001))/100*(1+0.001)
  profit = (100/group["Open"])*group["Open_t_1_test"]*(1-0.001)-100*(1+0.001)
  x = return_rate.sum()
  y = return_rate.mean()
  m = profit.sum()
  n = profit.mean()
  return x,y,m,n

# group the DataFrame by stock symbol and calculate the accuracy for each group
profit_df = prediction_1[['stock_symbol','Prediction','Open','Open_t_1_test']].groupby('stock_symbol').apply(calculate_profit).reset_index(name='profit')

# Extract the x and y values from the 'profit' column
profit_df[['return rate sum', 'return rate mean','profit sum','profit mean']] = profit_df['profit'].apply(pd.Series)

# Drop the 'profit' column
profit_df = profit_df.drop('profit', axis=1)

# print the resulting DataFrame
#profit_df

In [58]:
profit_df['profit sum'].sum()

34.93589876395693

In [59]:
def calculate_accuracy(group):
    return (group['Prediction'] == group['direction']).mean()

# group the DataFrame by stock symbol and calculate the accuracy for each group
accuracy_df = test1[['stock_symbol','direction','Prediction']].groupby('stock_symbol').apply(calculate_accuracy).reset_index(name='Accuracy')

# print the resulting DataFrame
accuracy_df

,stock_symbol,Accuracy
0,AAPL,0.392857
1,AMGN,0.678571
2,AXP,0.642857
3,BA,0.464286
4,CAT,0.464286
5,CRM,0.678571
6,CSCO,0.392857
7,CVX,0.535714
8,DIS,0.642857
9,DOW,0.571429


In [60]:
accuracy_df.Accuracy.mean()

0.5246305418719213

In [61]:
# define a function to calculate the precision for a group of rows
def calculate_precision(group, label):
    tp = ((group['Prediction'] == label) & (group['direction'] == label)).sum()
    fp = ((group['Prediction'] == label) & (group['direction'] != label)).sum()
    if (tp + fp) == 0:
        return 0
    else:
        return tp / (tp + fp)

# group the DataFrame by stock symbol and calculate the precision for each group
precision_0 = test1[['stock_symbol','direction','Prediction']].groupby('stock_symbol').apply(lambda x: calculate_precision(x, 0)).reset_index(name='Precision_0')
precision_1 = test1[['stock_symbol','direction','Prediction']].groupby('stock_symbol').apply(lambda x: calculate_precision(x, 1)).reset_index(name='Precision_1')

# merge the precision DataFrames with the accuracy DataFrame
result_df = pd.merge(accuracy_df, precision_0, on='stock_symbol').merge(precision_1, on='stock_symbol')

# print the resulting DataFrame
result_df

,stock_symbol,Accuracy,Precision_0,Precision_1
0,AAPL,0.392857,0.500000,0.363636
1,AMGN,0.678571,0.785714,0.571429
2,AXP,0.642857,0.611111,0.700000
3,BA,0.464286,0.384615,0.533333
4,CAT,0.464286,0.388889,0.600000
5,CRM,0.678571,0.700000,0.625000
6,CSCO,0.392857,0.375000,0.400000
7,CVX,0.535714,0.562500,0.500000
8,DIS,0.642857,0.642857,0.000000
9,DOW,0.571429,0.533333,0.615385


In [62]:
result_df.Precision_0.mean()

0.5065500280554518

In [63]:
result_df.Precision_1.mean()

0.5021004466157662

In [64]:
result_df.Accuracy.mean()

0.5246305418719213

In [65]:
final_result = result_df.merge(profit_df, on=['stock_symbol'], how='left')

In [66]:
final_result

,stock_symbol,Accuracy,Precision_0,Precision_1,return rate sum,return rate mean,profit sum,profit mean
0,AAPL,0.392857,0.500000,0.363636,-0.231834,-0.010538,-23.160265,-1.052739
1,AMGN,0.678571,0.785714,0.571429,0.065607,0.004686,6.554133,0.468152
2,AXP,0.642857,0.611111,0.700000,0.052823,0.005282,5.277032,0.527703
3,BA,0.464286,0.384615,0.533333,0.135868,0.009058,13.573208,0.904881
4,CAT,0.464286,0.388889,0.600000,-0.002630,-0.000263,-0.262720,-0.026272
5,CRM,0.678571,0.700000,0.625000,0.018540,0.002318,1.852189,0.231524
6,CSCO,0.392857,0.375000,0.400000,-0.060388,-0.003019,-6.032808,-0.301640
7,CVX,0.535714,0.562500,0.500000,0.019679,0.001640,1.965933,0.163828
8,DIS,0.642857,0.642857,0.000000,NaN,NaN,NaN,NaN
9,DOW,0.571429,0.533333,0.615385,0.036301,0.002792,3.626465,0.278959


In [67]:
test1.to_csv('drive/My Drive/thesis/dataset/final_prediction_dataset/ [New] final_prediction_separate_model [open to open] - price+macd+twitter+stocktwit.csv')

In [68]:
final_result.to_csv('drive/My Drive/thesis/dataset/final_prediction/ [New] final_prediction_separate_models [open to open] - price+macd+twitter+stocktwit.csv')